
# First - get player_id       

# url = http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season=2016&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id=112&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50



# Second - use player_id to get gamelog

# url = http://m.cubs.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id=592178&season=2016&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27

http://m.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id=493128&season=2016&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27


# need http://m.[team_name].mlb.com......  player_id=[player_id],season=[year],


In [ ]:
import os
import csv
import requests as r
import json

#NLCentral = ['112','138','134','158','113']  #Chicago Cubs,St. Louis Cardinals,Pittsburgh Pirates,Milwaukee Brewers,Cincinnati Reds
NLCentral = {'112':'Chicago Cubs',
             '138':'St. Louis Cardinals',
             '134':'Pittsburgh Pirates',
             '158':'Milwaukee Brewers',
             '113':'Cincinnati Reds'
            }

for team_id in NLCentral:
    
    team_path = 'E:/BigData/python/mlbCrawler/{}'.format(NLCentral[team_id])
    if not os.path.exists(team_path):
        os.makedirs(team_path)
    
    for season in range(2016,1999,-1):
        resToGetPlayerID = r.get('http://mlb.mlb.com/pubajax/wf/flow/stats.splayer?season={}&sort_order=%27desc%27&sort_column=%27avg%27&stat_type=hitting&page_type=SortablePlayer&team_id={}&game_type=%27R%27&player_pool=ALL&season_type=ANY&league_code=%27NL%27&sport_code=%27mlb%27&results=1000&recSP=1&recPP=50'.format(season,team_id))

        playerStats = json.loads(resToGetPlayerID.text)['stats_sortable_player']['queryResults']['row']

        # for playerNameID in playerStat:
        #         print(playerNameID['name_display_first_last'])

        player_ids=[]
        palyerIdName={}
        for playerID in playerStats:
                player_id = playerID['player_id']
                palyerIdName[playerID['player_id']] = playerID['name_display_first_last']
                #print(player_ids)
                player_ids.append(player_id)

        for id in player_ids:
            resToGetGameLog = r.get('http://m.mlb.com/lookup/json/named.sport_hitting_game_log_composed.bam?game_type=%27R%27&league_list_id=%27mlb%27&player_id={}&season={}&sit_code=%271%27&sit_code=%272%27&sit_code=%273%27&sit_code=%274%27&sit_code=%275%27&sit_code=%276%27&sit_code=%277%27&sit_code=%278%27&sit_code=%279%27&sit_code=%2710%27&sit_code=%2711%27&sit_code=%2712%27'.format(id,season))
            gameLog = json.loads(resToGetGameLog.text)['sport_hitting_game_log_composed']['sport_hitting_game_log']['queryResults']['row']
            #gameSize = json.loads(resToGetGameLog.text)['sport_hitting_game_log_composed']['sport_hitting_game_log']['queryResults']['totalSize']

            season_path = team_path + '/{}'.format(season)
            if not os.path.exists(season_path):
                os.makedirs(season_path)
            try:
                fieldnames = list((gameLog[0]).keys())
                with open(season_path + '/{}.csv'.format(palyerIdName[id]),'w') as csvfile:

                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    writer.writeheader()

                    for row in gameLog:
                        writer.writerow(row)
            except KeyError:
                with open(team_path + '/except_player.txt','w') as text_file:
                    print(str(season) + ':' + id + ' : ' + palyerIdName[id])
                    text_file.write(str(season) + ':' + id + ':' + palyerIdName[id]) 